In [2]:
# !pip install -U -q PyDrive
!pip install -q --upgrade ipython==5.5.0
!pip uninstall ipykernal


In [3]:
!pip install -q --upgrade ipykernel==4.6.0

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
# from pydrive.auth import GoogleAuth

# from pydrive.drive import GoogleDrive

# from google.colab import auth

# from oauth2client.client import GoogleCredentials 


# # 1. Authenticate and create the PyDrive client. 

# auth.authenticate_user()

# gauth = GoogleAuth()

# gauth.credentials = GoogleCredentials.get_application_default()

# drive = GoogleDrive(gauth)


In [4]:
!pip uninstall torch

Uninstalling torch-1.3.1:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/lib/python3.6/dist-packages/caffe2/*
    /usr/local/lib/python3.6/dist-packages/torch-1.3.1.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torch/*
Proceed (y/n)? y
  Successfully uninstalled torch-1.3.1


In [5]:
!pip install torch==0.3.1

     |████████████████████████████████| 496.4MB 32kB/s 
ERROR: torchvision 0.4.2 has requirement torch==1.3.1, but you'll have torch 0.3.1 which is incompatible.
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.


In [6]:
!pip uninstall torchvision

Uninstalling torchvision-0.4.2:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.4.2.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.4.2


In [10]:
!pip install torchvision==0.2.1

In [11]:
!pip install torchnet

In [1]:
import torch

torch.__version__

'0.3.1'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
cd /content/drive/'My Drive'/gestures

/content/drive/My Drive/gestures


In [4]:
!ls

capsnet_gestures_ver-Copy4.ipynb  data	epoch


In [5]:
!pip install SimpleITK

     |████████████████████████████████| 42.5MB 75kB/s 


In [0]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt
import cv2

In [0]:
BATCH_SIZE = 1
NUM_CLASSES = 5
NUM_EPOCHS = 200
NUM_ROUTING_ITERATIONS = 3

In [0]:

import SimpleITK as sitk
import numpy as np
import torch, os, glob, tqdm
import matplotlib.pyplot as plt
%matplotlib inline


# os.environ["CUDA_VISIBLE_DEVICES"]="0"


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from visdom import Visdom
from sklearn.model_selection import train_test_split

In [0]:
def find_classes(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx

def make_dataset(dir, class_to_idx):
    images = []
    dir = os.path.expanduser(dir)
    for target in sorted(os.listdir(dir)):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue

        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                item = (path, class_to_idx[target])
                images.append(item)

    return images

def reader(path):
    img =sitk.GetArrayFromImage(sitk.ReadImage(path))
    #img = cv2.resize(img, (224,224))
    #img = np.expand_dims(img, axis=0)
    #img = np.concatenate([img,img,img], axis=0).transpose((1,2,0)).astype(np.float32)
    return img
def resize_img(img):
    gray_img = img.transpose((2,0,1))[0]
    resize_gray_img = cv2.resize(gray_img, (160,120))
    return resize_gray_img    

In [0]:
class DicomFolder(Dataset):
    def __init__(self, root, transform=None):
        classes, class_to_idx = find_classes(root)
        imgs = make_dataset(root, class_to_idx)
        
        if len(imgs)==0:
            raise(RuntimeError("Found 0 images in subfolders of: " + root + "\n"))
        
        self.root = root
        self.imgs = imgs
        self.classes = classes
        self.class_to_idx = class_to_idx
        self.transform = transform
        
    def __getitem__(self, index):
        
        path, label = self.imgs[index]
        
        img = reader(path)
        resized_img = resize_img(img)
        #dicom=cv2.resize(dicom,(160,120))
       
        #dicom = np.expand_dims(cv2.resize(self.loader(path)[0], (224,224)), axis=2).astype(np.float32)
        
        return resized_img, label
        
        
    def __len__(self):
        return len(self.imgs)

In [11]:
torch.eye(NUM_CLASSES)[3].type(torch.LongTensor).view(1,-1)


 0  0  0  1  0
[torch.LongTensor of size 1x5]

In [12]:
train_dset = DicomFolder('/content/drive/My Drive/gestures/data/01except/')
val_dset = DicomFolder('/content/drive/My Drive/gestures/data/01/')
dsets = {'train':train_dset, 'val':val_dset}
dset_loaders = {x: DataLoader(dsets[x], batch_size=1,
                            shuffle=True, num_workers=4) for x in ['train', 'val']}
dset_sizes = {x: len(dsets[x]) for x in ['train', 'val']}
dset_classes = dsets['train'].classes
dset_classes

['c', 'g', 'h', 'o', 't']

In [13]:
test_dset = DicomFolder('/content/drive/My Drive/gestures/data/01/')
test_dsets = {'test':test_dset}
test_loaders = {x: DataLoader(test_dsets[x], batch_size=1,
                            shuffle=True, num_workers=4) for x in ['test']}
test_sizes = {x: len(test_dsets[x]) for x in ['test']}
test_classes = test_dsets['test'].classes
test_classes
# test_sizes

['c', 'g', 'h', 'o', 't']

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np


def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)))
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)


def augmentation(x, max_shift=2):
    _, _, height, width = x.size()

    h_shift, w_shift = np.random.randint(-max_shift, max_shift + 1, size=2)
    source_height_slice = slice(max(0, h_shift), h_shift + height)
    source_width_slice = slice(max(0, w_shift), w_shift + width)
    target_height_slice = slice(max(0, -h_shift), -h_shift + height)
    target_width_slice = slice(max(0, -w_shift), -w_shift + width)

    shifted_image = torch.zeros(*x.size())
    shifted_image[:, :, source_height_slice, source_width_slice] = x[:, :, target_height_slice, target_width_slice]
    return shifted_image.float()


class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]

            logits = Variable(torch.zeros(*priors.size())).cuda()
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
                    logits = logits + delta_logits
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs


class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()

#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=9, stride=1)
#         self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
#                                              kernel_size=9, stride=2)
#         self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=32 * 52 *72, in_channels=8,
#                                            out_channels=16)

        
        
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=9, stride=1)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=32 * 52 *72, in_channels=8,
                                           out_channels=16)
        
        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 2048),
            nn.ReLU(inplace=True),
            nn.Linear(2048, 120*160),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        x = F.relu(self.conv1(x), inplace=True)
        
       
        x = self.primary_capsules(x)
        #print(x.size())
        z = self.digit_capsules(x)
        #print(z.size())
        x=z.squeeze().unsqueeze(0)
        #x=z.squeeze().transpose(0, 1)
        #print(x.size())
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes)
        #print(classes.size())
        if y is None:
            # In all batches, get the most active capsule.
            _, max_length_indices = classes.max(dim=1)


##수정
            masked = Variable(torch.sparse.torch.eye(NUM_CLASSES))

            if torch.cuda.is_available():
                y = masked.cuda().index_select(dim=0, index=max_length_indices.data)

            # if torch.cuda.is_available():
            #     y = Variable(torch.sparse.torch.eye(NUM_CLASSES)).cuda().index_select(dim=0, index=max_length_indices.data)
            # else:
            #     y = Variable(torch.sparse.torch.eye(NUM_CLASSES)).index_select(dim=0, index=max_length_indices.data)

  
        reconstructions = self.decoder((x * y[:, :, None]).view(x.size(0), -1))
        #print(reconstructions.size())
  
        return classes, reconstructions


class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()
        self.reconstruction_loss = nn.MSELoss(size_average=False)

    def forward(self, images, labels, classes, reconstructions):
        left = F.relu(0.9 - classes, inplace=True) ** 2
        right = F.relu(classes - 0.1, inplace=True) ** 2

        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()

        reconstruction_loss = self.reconstruction_loss(reconstructions, images)

        return (margin_loss + 0.0005 * reconstruction_loss) / images.size(0)

In [21]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt

model = CapsuleNet()
# model.load_state_dict(torch.load('epochs/epoch_327.pt'))
model.cuda()

print("# parameters:", sum(param.numel() for param in model.parameters()))

optimizer = Adam(model.parameters())

engine = Engine()
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(NUM_CLASSES, normalized=True)

# train_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer Train Loss'})
# train_error_logger = VisdomPlotLogger('line', opts={'title': '120160_layer Train Accuracy'})
# test_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer Test Loss'})
# test_accuracy_logger = VisdomPlotLogger('line', opts={'title': '120160_layer Test Accuracy'})
# confusion_logger = VisdomLogger('heatmap', opts={'title': '120160_layer Confusion matrix',
#                                                  'columnnames': list(range(NUM_CLASSES)),
#                                                  'rownames': list(range(NUM_CLASSES))})
# ground_truth_logger = VisdomLogger('image', opts={'title': '120160_layer Ground Truth'})
# reconstruction_logger = VisdomLogger('image', opts={'title': '120160_layer Reconstruction'})

capsule_loss = CapsuleLoss()


def get_iterator(mode):
    
    #dataset = MNIST(root='./data', download=True, train=mode)
    #data = getattr(dataset, 'train_data' if mode else 'test_data')
    #labels = getattr(dataset, 'train_labels' if mode else 'test_labels')

    d,l =  next(iter(dset_loaders['train']))
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


def processor(sample):
    data, labels, training = sample

    data = augmentation(data.unsqueeze(1).float() / 255.0)
    labels = torch.LongTensor(labels)

    labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

    data = Variable(data).cuda()
    labels = Variable(labels).cuda()

    if training:
        classes, reconstructions = model(data, labels)
    else:
        classes, reconstructions = model(data)

    loss = capsule_loss(data, labels, classes, reconstructions)

    return loss, classes


def reset_meters():
    meter_accuracy.reset()
    meter_loss.reset()
    confusion_meter.reset()


def on_sample(state):
    state['sample'].append(state['train'])


def on_forward(state):
    meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    #print(state['output'].data.size())
    #print(torch.LongTensor(state['sample'][1]).size())
    one_hot = torch.eye(NUM_CLASSES)[torch.LongTensor(state['sample'][1])].type(torch.LongTensor).view(1,-1)
    #print('onehot',one_hot)
    #confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    confusion_meter.add(state['output'].data, one_hot)
    
    meter_loss.add(state['loss'].data[0])


def on_start_epoch(state):
    reset_meters()
    state['iterator'] = tqdm(state['iterator'])


def on_end_epoch(state):
    print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    # train_loss_logger.log(state['epoch'], meter_loss.value()[0])
    # train_error_logger.log(state['epoch'], meter_accuracy.value()[0])

    reset_meters()

    #engine.test(processor, get_iterator(False))
    engine.test(processor, dset_loaders['val'])    

    # test_loss_logger.log(state['epoch'], meter_loss.value()[0])
    # test_accuracy_logger.log(state['epoch'], meter_accuracy.value()[0])
    # confusion_logger.log(confusion_meter.value())

    print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    torch.save(model.state_dict(), '/content/drive/My Drive/gestures/epoch/01_ep/epoch_%d.pt' % state['epoch'])

    # Reconstruction visualization.

    test_sample = next(iter(dset_loaders['val']))

    ground_truth = (test_sample[0].unsqueeze(1).float() / 255.0)
    _, reconstructions = model(Variable(ground_truth).cuda())
    reconstruction = reconstructions.cpu().view_as(ground_truth).data

    # ground_truth_logger.log(
    #     make_grid(ground_truth, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())
    # reconstruction_logger.log(
    #     make_grid(reconstruction, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())

# def on_start(state):
#     state['epoch'] = 327
#
# engine.hooks['on_start'] = on_start
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, dset_loaders['train'], maxepoch=NUM_EPOCHS, optimizer=optimizer)

  0%|          | 0/795 [00:00<?, ?it/s]

# parameters: 123529728


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:109: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
100%|██████████| 795/795 [01:52<00:00,  7.02it/s]

[Epoch 1] Training Loss: 0.6801 (Accuracy: 23.14%)


RuntimeError: ignored

inference

In [0]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt

model = CapsuleNet()

model.load_state_dict(torch.load('/data2/ar/gestures20170903/epoch/200250/epoch_198.pt'))
model.cuda()

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

In [0]:
print("# parameters:", sum(param.numel() for param in model.parameters()))

optimizer = Adam(model.parameters())
engine = Engine()
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(NUM_CLASSES, normalized=True)

train_loss_logger = VisdomPlotLogger('line', opts={'title': 'Train Loss'})
train_error_logger = VisdomPlotLogger('line', opts={'title': 'Train Accuracy'})
test_loss_logger = VisdomPlotLogger('line', opts={'title': 'Test Loss'})
test_accuracy_logger = VisdomPlotLogger('line', opts={'title': 'Test Accuracy'})
confusion_logger = VisdomLogger('heatmap', opts={'title': 'Confusion matrix',
                                                 'columnnames': list(range(NUM_CLASSES)),
                                                 'rownames': list(range(NUM_CLASSES))})
ground_truth_logger = VisdomLogger('image', opts={'title': 'Ground Truth'})
reconstruction_logger = VisdomLogger('image', opts={'title': 'Reconstruction'})

capsule_loss = CapsuleLoss()

# def test_processor(sample):
#     data, labels = sample

#     data = augmentation(data.unsqueeze(1).float() )
#     labels = torch.LongTensor(labels)

#     labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

#     data = Variable(data).cuda()
#     labels = Variable(labels).cuda()

# #     if training:
#     classes, reconstructions = model(data, labels)
# #     else:
# #         classes, reconstructions = model(data)

#     print(labels , classes)
#     loss = capsule_loss(data, labels, classes, reconstructions)

#     return loss, classes


def get_digit_caps(model, image):
    input_ = Variable(image.unsqueeze(0), volatile=True)
    
    digit_caps, probs = model.capsnet(input_)
    return digit_caps

# takes digit_caps output and target label
def get_reconstruction(model, digit_caps, label):
    target = Variable(torch.LongTensor([label]), volatile=True)
    reconstruction = model.reconstruction_net(digit_caps, target)
    return reconstruction.data.cpu().numpy()[0].reshape(28, 28)

# create reconstructions with perturbed digit capsule
def dimension_perturbation_reconstructions(model, digit_caps, label, dimension, dim_values):
    reconstructions = []
    for dim_value in dim_values:
        digit_caps_perturbed = digit_caps.clone()
        digit_caps_perturbed[0, label, dimension] = dim_value
        reconstruction = get_reconstruction(model, digit_caps_perturbed, label)
        reconstructions.append(reconstruction)
    return reconstructions

def test(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

#     engine.test(test_processor, test_loaders['test'])  
    
    data, labels = test_loaders['test']
    data = augmentation(data.unsqueeze(1).float() )
    labels = torch.LongTensor(labels)

    labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

    data1 = Variable(data).cuda()
    labels = Variable(labels).cuda()
    
    y_pred, x_recon = model(data1)
    
    test_loss +=capsule_loss(data1, labels, y_pred, x_recon)
    y_pred = y_pred.data.max(1)[1]
    y_true = data1.data.max(1)[1]
    correct += y_pred.eq(y_true).cpu().sum()
    test_loss /= len(test_loader.dataset)
    return test_loss, correct / len(test_loader.dataset)        

test_acc=[]
test_loss=[]
# def test_processor(sample):
#     test_loss = 0
#     correct = 0
#     data, labels= sample

#     data = augmentation(data.unsqueeze(1).float() )
#     labels = torch.LongTensor(labels)

#     labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

#     data = Variable(data).cuda()
#     labels = Variable(labels).cuda()

# #     if training:
#     classes, reconstructions = model(data, labels)
# #     else:
# #         classes, reconstructions = model(data)

#     print(labels , classes)
#     loss = capsule_loss(data, labels, classes, reconstructions)
#     y_pred = classes.data.max(1)[1]
#     y_true = data.data.max(1)[1]

    
#     correct += y_pred.eq(y_true).cpu().sum()
#     test_loss /= len(test_loaders['test'])
#     print('test acc = %.4f, test loss = %.5f' % (correct/len(test_loaders['test']), test_loss))
# #     print('test acc = %.4f' % (test_acc))
    
#     return loss, classes
running_loss = 0.0
corr=[]
criterion = nn.CrossEntropyLoss()
criterion.cuda()
def test_processor(sample):
    running_loss = 0.0
    running_corrects = 0
    inputs, labels = sample
    
    inputs = inputs.unsqueeze(1).float()
    labels = torch.LongTensor(labels)
    labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)
    
    labels, inputs  = Variable(labels.float().cuda()), Variable(inputs.float().cuda())
    optimizer.zero_grad()
    
    classes, reconstructions = model(inputs)
    
    loss = capsule_loss(inputs, labels, classes, reconstructions)
    _, preds = torch.max(classes.data, 1)
    _, labb = torch.max(labels,1)
#     loss = criterion(classes, labels)
    running_loss += loss.data[0]

    running_corrects += torch.sum(preds.cpu() == labb.cpu().data)
    
    corr.append(running_corrects)
    epoch_loss = running_loss / len(test_loaders['test'])
    epoch_acc = float(running_corrects) / len(test_loaders['test'])
    print(' Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc*100))
    print (sum(corr))
    return epoch_loss, epoch_acc

In [0]:
# loss,classes = test_processor(test_loaders['test'])
engine.test(test_processor, test_loaders['test'])

# print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
#     '195', meter_loss.value()[0], meter_accuracy.value()[0]))
# test_loss, test_acc = test(model=model, test_loader=test_loaders['test'])

# show_reconstruction(model, test_loader, 50)

In [0]:
len(test_dsets['test'])

In [0]:
engine.test(test_processor, test_loaders['test'])    

test_state=torch.load('/data2/ar/gestures20170903/epoch/120160_layer/epoch_198.pt')
test_loss_logger.log(test_state, meter_loss.value()[0])
test_accuracy_logger.log(test_state, meter_accuracy.value()[0])
confusion_logger.log(confusion_meter.value())

print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
    test_state, meter_loss.value()[0], meter_accuracy.value()[0]))


In [0]:
    d,l =  next(iter(dset_loaders['train']))
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


In [0]:
# Get reconstructions
images = []
reconstructions = []
for i in range(8):
    image_tensor, label = dataset[i]
    digit_caps = get_digit_caps(model, image_tensor)
    reconstruction = get_reconstruction(model, digit_caps, label)
    images.append(image_tensor.numpy()[0])
    reconstructions.append(reconstruction)

In [0]:
# Plot reconstructions
fig, axs = plt.subplots(2, 8, figsize=(16, 4))
axs[0, 0].set_ylabel('Org image', size='large')
axs[1, 0].set_ylabel('Reconstruction', size='large')
for i in range(8):
    axs[0, i].imshow(images[i], cmap='gray')
    axs[1, i].imshow(reconstructions[i], cmap='gray')
    axs[0, i].set_yticks([])
    axs[0, i].set_xticks([])
    axs[1, i].set_yticks([])
    axs[1, i].set_xticks([])

In [0]:
test_dset = DicomFolder('/content/drive/'My Drive'/gestures/data/01/')
test_dsets = {'test':test_dset}
test_loaders = {x: DataLoader(test_dsets[x], batch_size=1,
                            shuffle=True, num_workers=4) for x in ['test']}
test_sizes = {x: len(test_dsets[x]) for x in ['test']}
test_classes = dsets['train'].classes
test_classes

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
import numpy as np


def softmax(input, dim=1):
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(transposed_input.contiguous().view(-1, transposed_input.size(-1)))
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)


def augmentation(x, max_shift=2):
    _, _, height, width = x.size()

    h_shift, w_shift = np.random.randint(-max_shift, max_shift + 1, size=2)
    source_height_slice = slice(max(0, h_shift), h_shift + height)
    source_width_slice = slice(max(0, w_shift), w_shift + width)
    target_height_slice = slice(max(0, -h_shift), -h_shift + height)
    target_width_slice = slice(max(0, -w_shift), -w_shift + width)

    shifted_image = torch.zeros(*x.size())
    shifted_image[:, :, source_height_slice, source_width_slice] = x[:, :, target_height_slice, target_width_slice]
    return shifted_image.float()


class CapsuleLayer(nn.Module):
    def __init__(self, num_capsules, num_route_nodes, in_channels, out_channels, kernel_size=None, stride=None,
                 num_iterations=NUM_ROUTING_ITERATIONS):
        super(CapsuleLayer, self).__init__()

        self.num_route_nodes = num_route_nodes
        self.num_iterations = num_iterations

        self.num_capsules = num_capsules

        if num_route_nodes != -1:
            self.route_weights = nn.Parameter(torch.randn(num_capsules, num_route_nodes, in_channels, out_channels))
        else:
            self.capsules = nn.ModuleList(
                [nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=0) for _ in
                 range(num_capsules)])

    def squash(self, tensor, dim=-1):
        squared_norm = (tensor ** 2).sum(dim=dim, keepdim=True)
        scale = squared_norm / (1 + squared_norm)
        return scale * tensor / torch.sqrt(squared_norm)

    def forward(self, x):
        if self.num_route_nodes != -1:
            priors = x[None, :, :, None, :] @ self.route_weights[:, None, :, :, :]

            logits = Variable(torch.zeros(*priors.size())).cuda()
            for i in range(self.num_iterations):
                probs = softmax(logits, dim=2)
                outputs = self.squash((probs * priors).sum(dim=2, keepdim=True))

                if i != self.num_iterations - 1:
                    delta_logits = (priors * outputs).sum(dim=-1, keepdim=True)
                    logits = logits + delta_logits
        else:
            outputs = [capsule(x).view(x.size(0), -1, 1) for capsule in self.capsules]
            outputs = torch.cat(outputs, dim=-1)
            outputs = self.squash(outputs)

        return outputs


class CapsuleNet(nn.Module):
    def __init__(self):
        super(CapsuleNet, self).__init__()

#         self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=9, stride=1)
#         self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=32,
#                                              kernel_size=9, stride=2)
#         self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=32 * 52 *72, in_channels=8,
#                                            out_channels=16)

        
        
        
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=256, kernel_size=9, stride=1)
        self.primary_capsules = CapsuleLayer(num_capsules=8, num_route_nodes=-1, in_channels=256, out_channels=128,
                                             kernel_size=9, stride=2)
        self.primary_capsules2 = CapsuleLayer(num_capsules=8, num_route_nodes=128*52*72, in_channels=128, out_channels=32,
                                             kernel_size=9, stride=2)
        self.digit_capsules = CapsuleLayer(num_capsules=NUM_CLASSES, num_route_nodes=32 * 22*32, in_channels=8,
                                           out_channels=16)
        
        self.decoder = nn.Sequential(
            nn.Linear(16 * NUM_CLASSES, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 120*160),
            nn.Sigmoid()
        )

    def forward(self, x, y=None):
        x = F.relu(self.conv1(x), inplace=True)
        
       
        x = self.primary_capsules(x)
        #print(x.size())
        x = self.primary_capsules2(x)
        z = self.digit_capsules(x)
        #print(z.size())
        x=z.squeeze().unsqueeze(0)
        #x=z.squeeze().transpose(0, 1)
        #print(x.size())
        classes = (x ** 2).sum(dim=-1) ** 0.5
        classes = F.softmax(classes)
        #print(classes.size())
        if y is None:
            # In all batches, get the most active capsule.
            _, max_length_indices = classes.max(dim=1)
            y = Variable(torch.sparse.torch.eye(NUM_CLASSES)).cuda().index_select(dim=0, index=max_length_indices.data)
  
        reconstructions = self.decoder((x * y[:, :, None]).view(x.size(0), -1))
        #print(reconstructions.size())
  
        return classes, reconstructions


class CapsuleLoss(nn.Module):
    def __init__(self):
        super(CapsuleLoss, self).__init__()
        self.reconstruction_loss = nn.MSELoss(size_average=False)

    def forward(self, images, labels, classes, reconstructions):
        left = F.relu(0.9 - classes, inplace=True) ** 2
        right = F.relu(classes - 0.1, inplace=True) ** 2

        margin_loss = labels * left + 0.5 * (1. - labels) * right
        margin_loss = margin_loss.sum()

        reconstruction_loss = self.reconstruction_loss(reconstructions, images)

        return (margin_loss + 0.0005 * reconstruction_loss) / images.size(0)

In [0]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt

model = CapsuleNet()
# model.load_state_dict(torch.load('epochs/epoch_327.pt'))
model.cuda()

print("# parameters:", sum(param.numel() for param in model.parameters()))

optimizer = Adam(model.parameters())

engine = Engine()
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(NUM_CLASSES, normalized=True)

train_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Train Loss'})
train_error_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Train Accuracy'})
test_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Test Loss'})
test_accuracy_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Test Accuracy'})
confusion_logger = VisdomLogger('heatmap', opts={'title': '120160_layer2 Confusion matrix',
                                                 'columnnames': list(range(NUM_CLASSES)),
                                                 'rownames': list(range(NUM_CLASSES))})
ground_truth_logger = VisdomLogger('image', opts={'title': '120160_layer2 Ground Truth'})
reconstruction_logger = VisdomLogger('image', opts={'title': '120160_layer2 Reconstruction'})

capsule_loss = CapsuleLoss()


def get_iterator(mode):
    
    #dataset = MNIST(root='./data', download=True, train=mode)
    #data = getattr(dataset, 'train_data' if mode else 'test_data')
    #labels = getattr(dataset, 'train_labels' if mode else 'test_labels')

    d,l =  next(iter(dset_loaders['train']))
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


def processor(sample):
    data, labels, training = sample

    data = augmentation(data.unsqueeze(1).float() / 255.0)
    labels = torch.LongTensor(labels)

    labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

    data = Variable(data).cuda()
    labels = Variable(labels).cuda()

    if training:
        classes, reconstructions = model(data, labels)
    else:
        classes, reconstructions = model(data)

    loss = capsule_loss(data, labels, classes, reconstructions)

    return loss, classes


def reset_meters():
    meter_accuracy.reset()
    meter_loss.reset()
    confusion_meter.reset()


def on_sample(state):
    state['sample'].append(state['train'])


def on_forward(state):
    meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    #print(state['output'].data.size())
    #print(torch.LongTensor(state['sample'][1]).size())
    one_hot = torch.eye(NUM_CLASSES)[torch.LongTensor(state['sample'][1])].type(torch.LongTensor).view(1,-1)
    #print('onehot',one_hot)
    #confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    confusion_meter.add(state['output'].data, one_hot)
    
    meter_loss.add(state['loss'].data[0])


def on_start_epoch(state):
    reset_meters()
    state['iterator'] = tqdm(state['iterator'])


def on_end_epoch(state):
    print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    train_loss_logger.log(state['epoch'], meter_loss.value()[0])
    train_error_logger.log(state['epoch'], meter_accuracy.value()[0])

    reset_meters()

    #engine.test(processor, get_iterator(False))
    engine.test(processor, dset_loaders['val'])    

    test_loss_logger.log(state['epoch'], meter_loss.value()[0])
    test_accuracy_logger.log(state['epoch'], meter_accuracy.value()[0])
    confusion_logger.log(confusion_meter.value())

    print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    torch.save(model.state_dict(), '/home/secl00/env_tensor112/0_Segmentation/gestures/epoch/06_ep/epoch_%d.pt' % state['epoch'])

    # Reconstruction visualization.

    test_sample = next(iter(dset_loaders['val']))

    ground_truth = (test_sample[0].unsqueeze(1).float() / 255.0)
    _, reconstructions = model(Variable(ground_truth).cuda())
    reconstruction = reconstructions.cpu().view_as(ground_truth).data

    ground_truth_logger.log(
        make_grid(ground_truth, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())
    reconstruction_logger.log(
        make_grid(reconstruction, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())

# def on_start(state):
#     state['epoch'] = 327
#
# engine.hooks['on_start'] = on_start
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, dset_loaders['train'], maxepoch=NUM_EPOCHS, optimizer=optimizer)

Inference

In [0]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt

model = CapsuleNet()
# model.load_state_dict(torch.load('epochs/epoch_327.pt'))
model.cuda()

In [0]:
from torch.autograd import Variable
from torch.optim import Adam
from torchnet.engine import Engine
from torchnet.logger import VisdomPlotLogger, VisdomLogger
from torchvision.utils import make_grid
from torchvision.datasets.mnist import MNIST
from tqdm import tqdm
import torchnet as tnt

model = CapsuleNet()
# model.load_state_dict(torch.load('epochs/epoch_327.pt'))
model.cuda()

print("# parameters:", sum(param.numel() for param in model.parameters()))

optimizer = Adam(model.parameters())

engine = Engine()
meter_loss = tnt.meter.AverageValueMeter()
meter_accuracy = tnt.meter.ClassErrorMeter(accuracy=True)
confusion_meter = tnt.meter.ConfusionMeter(NUM_CLASSES, normalized=True)

train_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Train Loss'})
train_error_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Train Accuracy'})
test_loss_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Test Loss'})
test_accuracy_logger = VisdomPlotLogger('line', opts={'title': '120160_layer2 Test Accuracy'})
confusion_logger = VisdomLogger('heatmap', opts={'title': '120160_layer2 Confusion matrix',
                                                 'columnnames': list(range(NUM_CLASSES)),
                                                 'rownames': list(range(NUM_CLASSES))})
ground_truth_logger = VisdomLogger('image', opts={'title': '120160_layer2 Ground Truth'})
reconstruction_logger = VisdomLogger('image', opts={'title': '120160_layer2 Reconstruction'})

capsule_loss = CapsuleLoss()


def get_iterator(mode):
    
    #dataset = MNIST(root='./data', download=True, train=mode)
    #data = getattr(dataset, 'train_data' if mode else 'test_data')
    #labels = getattr(dataset, 'train_labels' if mode else 'test_labels')

    d,l =  next(iter(dset_loaders['train']))
    tensor_dataset = tnt.dataset.TensorDataset([data, labels])

    return tensor_dataset.parallel(batch_size=BATCH_SIZE, num_workers=4, shuffle=mode)


def processor(sample):
    data, labels, training = sample

    data = augmentation(data.unsqueeze(1).float() / 255.0)
    labels = torch.LongTensor(labels)

    labels = torch.sparse.torch.eye(NUM_CLASSES).index_select(dim=0, index=labels)

    data = Variable(data).cuda()
    labels = Variable(labels).cuda()

    if training:
        classes, reconstructions = model(data, labels)
    else:
        classes, reconstructions = model(data)

    loss = capsule_loss(data, labels, classes, reconstructions)

    return loss, classes


def reset_meters():
    meter_accuracy.reset()
    meter_loss.reset()
    confusion_meter.reset()


def on_sample(state):
    state['sample'].append(state['train'])


def on_forward(state):
    meter_accuracy.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    #print(state['output'].data.size())
    #print(torch.LongTensor(state['sample'][1]).size())
    one_hot = torch.eye(NUM_CLASSES)[torch.LongTensor(state['sample'][1])].type(torch.LongTensor).view(1,-1)
    #print('onehot',one_hot)
    #confusion_meter.add(state['output'].data, torch.LongTensor(state['sample'][1]))
    confusion_meter.add(state['output'].data, one_hot)
    
    meter_loss.add(state['loss'].data[0])


def on_start_epoch(state):
    reset_meters()
    state['iterator'] = tqdm(state['iterator'])


def on_end_epoch(state):
    print('[Epoch %d] Training Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    train_loss_logger.log(state['epoch'], meter_loss.value()[0])
    train_error_logger.log(state['epoch'], meter_accuracy.value()[0])

    reset_meters()

    #engine.test(processor, get_iterator(False))
    engine.test(processor, dset_loaders['val'])    

    test_loss_logger.log(state['epoch'], meter_loss.value()[0])
    test_accuracy_logger.log(state['epoch'], meter_accuracy.value()[0])
    confusion_logger.log(confusion_meter.value())

    print('[Epoch %d] Testing Loss: %.4f (Accuracy: %.2f%%)' % (
        state['epoch'], meter_loss.value()[0], meter_accuracy.value()[0]))

    torch.save(model.state_dict(), '/data2/ar/gestures20170903/epoch/120160_layer2/epoch_%d.pt' % state['epoch'])

    # Reconstruction visualization.

    test_sample = next(iter(dset_loaders['val']))

    ground_truth = (test_sample[0].unsqueeze(1).float() / 255.0)
    _, reconstructions = model(Variable(ground_truth).cuda())
    reconstruction = reconstructions.cpu().view_as(ground_truth).data

    ground_truth_logger.log(
        make_grid(ground_truth, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())
    reconstruction_logger.log(
        make_grid(reconstruction, nrow=int(BATCH_SIZE ** 0.5), normalize=True, range=(0, 1)).numpy())

# def on_start(state):
#     state['epoch'] = 327
#
# engine.hooks['on_start'] = on_start
engine.hooks['on_sample'] = on_sample
engine.hooks['on_forward'] = on_forward
engine.hooks['on_start_epoch'] = on_start_epoch
engine.hooks['on_end_epoch'] = on_end_epoch

engine.train(processor, dset_loaders['train'], maxepoch=NUM_EPOCHS, optimizer=optimizer)